# Import packages

In [1]:
from array import array
import itertools
import time
import math

from matplotlib.ticker import AutoMinorLocator, MultipleLocator

import matplotlib.pyplot as plt

import sys
sys.path.insert(0,'/usr/common/software/rootpy/')
import root_numpy
import numpy as np


#import astroML.plotting as aml
#import iminuit
import pandas as pd
import root_pandas as rpd
from root_pandas import read_root
import ROOT
from ROOT import TH1D
from matplotlib.ticker import NullFormatter

Welcome to ROOTaaS 6.06/06


# File

In [2]:
#RootFile = '13d_4L_newphi.root'
#RootFile = '13d_4L_MB_newphi_0GeVTracks.root'
#RootFile = '13f_4L_MB_0GeVTracks.root'
#RootFile = '13f_4L_GMB_ratio.root'
RootFile = '13d_4L_SE_GMB_Ratio.root'
#RootFile = '13e_4L_SE_GMB_Ratio.root'

# Define Bins

In [3]:
#FIXME: Read from Corr_config.yaml
zTbins = [0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2]
#pTbins = [10, 11, 12.5, 16]
pTbins = [0, 0, 10, 16]
pTbins = [0, 0, 12, 15]

# Plot Titles

In [4]:
def Plot1D (histo, ptbin, ztbin,DNN_Bool):
    histo.SetTitle("Same/Mix: p_{T}^{Clus.}(%1.1f-%1.1f),  z_{T}(%1.2f-%1.2f)"
                   %(pTbins[ptbin],pTbins[ptbin+1],zTbins[ztbin],zTbins[ztbin+1]))
    histo.GetXaxis().SetTitle("#Delta#eta")
    histo.GetYaxis().SetTitle("#frac{1}{N_{Trig}}#frac{d^{2}N}{d#phid#eta}")
    histo.GetYaxis().SetTitleOffset(1)
    ROOT.gStyle.SetOptStat("")
    if not(DNN_Bool):
        histo.SetLineColor(2)

In [5]:
def Plot2D (histo2D,ptbin,ztbin,DNN_Bool):
    DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN))
    histo2D.GetXaxis().SetTitle("#Delta#phi")
    histo2D.GetYaxis().SetTitle("#Delta#eta")
    #histo2D.GetZaxis().SetTitle("#frac{1}{N_{Trig}}#frac{d^{2}N}{d#phid#eta}")
    histo2D.GetZaxis().SetTitle("#frac{d^{2}N}{d#phid#eta}")
    histo2D.SetTitle('#gamma-MB Mixed Events: DNN%i pT (%1.0f-%1.0f) zT (%1.2f-%1.2f)' 
    #histo2D.SetTitle('#gamma-hadron Events: DNN%i pT (%1.0f-%1.0f) zT (%1.2f-%1.2f)' 
                       %(DNN_Rgn,pTbins[ptbin],pTbins[ptbin+1],zTbins[ztbin],zTbins[ztbin+1]))
    #ROOT.gStyle.SetOptStat("")
    histo2D.GetYaxis().SetRangeUser(-1.2,1.2)
    histo2D.GetZaxis().SetTitleOffset(2)
    #ROOT.gStyle.SetOptStat("")

In [6]:
def Get_NTriggers(filename, ipt, Signal_DNN=True):
    DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN)) 
    file = ROOT.TFile(filename)
    ntrig_histo = file.Get('N_Triggers_pT%1.0f_%1.0f' %(pTbins[ipt],pTbins[ipt+1]))
    NTriggers = 1
    if not(ntrig_histo == None):
        NTriggers = ntrig_histo.GetEntries()
    file.Close()
    return NTriggers

# 2D Correlations

In [7]:
def Get2D(filename, ipt, izt, Signal_DNN=True):

    DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN)) #convert bool to DNN_1 (Sgn) or DNN_2 (Bkgd)
    file = ROOT.TFile(filename)
    histoDNN = file.Get('DNN%i_Correlation__pT%1.0f_%1.0f__zT%1.0f_zT%1.0f' 
                       %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1],10*zTbins[izt],10*zTbins[izt+1]))
    
    #histoDNN = file.Get('Correlation__pT%1.0f_%1.0f__zT%1.0f_zT%1.0f' 
    #                   %(pTbins[ipt],pTbins[ipt+1],10*zTbins[izt],10*zTbins[izt+1]))
    
    histoDNN.SetTitle('#gamme-hadron [High DNN] pT (%1.0f-%1.0f) zT (%1.2f-%1.2f)' 
                       %(pTbins[ipt],pTbins[ipt+1],zTbins[izt],zTbins[izt+1]))
    
    histoDNN.RebinX(2)
    histoDNN.SetDirectory(0)
    Plot2D(histoDNN,ipt,izt,Signal_DNN)
    
    file.Close()
    return histoDNN

In [8]:
def Loop2D(filename,Signal_DNN=True):
    DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN))
    canvases = []
    for ipt in range(3):
        if(ipt > 0): continue
        canvases.append(ROOT.TCanvas("Canvas%i"%(ipt),"Canvas",6000,5000))
        canvases[ipt].Divide(2,2)
        for izt in range(4):
            #histoDNN = Get2D(filename,ipt,izt,Signal_DNN)
            histoDNN = Get2D(filename,ipt+2,izt,Signal_DNN)
            canvases[ipt].cd(izt+1)
            gpad1 = canvases[ipt].cd(izt+1)
            gpad1.SetLeftMargin(0.15)
            histoDNN.Draw("SURF2")
            canvases[ipt].SetLeftMargin(0.15)
    return canvases


In [ ]:
Signal_DNN = True
DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN))
print (DNN_Rgn)
canvases = Loop2D(RootFile,Signal_DNN)
ROOT.disableJSVis()
for ipt in range(3):
    if (ipt > 0): continue
    canvases[ipt].Draw()
    canvases[ipt].SaveAs('pics/2D_MIX_Correlations_CombinedGeV_DNN_%i.png'%(DNN_Rgn))

# Phi Projection

<sup>**NOTE**: The DNN corresponding to one photon is reflected in the naming scheme of histograms, where DNN1_Histo is signal (the same here) but background (DNN corresponding to 2 photons) is DNN2_Histo. Here, the DNN variable is a boolian corresponding to *true for signal*, *false for background*<sup>

In [ ]:
def GetPhiProj(filename, ipt, izt, Signal_DNN=True):

    DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN)) #convert bool to DNN_1 (Sgn) or DNN_2 (Bkgd)
    file = ROOT.TFile(filename)
    histo2D = file.Get('DNN%i_Correlation__pT%1.0f_%1.0f__zT%1.0f_zT%1.0f' 
                       %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1],10*zTbins[izt],10*zTbins[izt+1]))
    #integral
    Eta_Axis = histo2D.GetYaxis()
    PhiProjection = histo2D.ProjectionX('DNN%i_PhiProjection__pT_%1.0f_%1.0f__zt_%1.0f_%1.0f' 
                                  %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1],10*zTbins[izt],
                                    10*zTbins[izt+1]),Eta_Axis.FindBin(-0.6),Eta_Axis.FindBin(0.6));
    PhiProjection.SetDirectory(0)
    PhiProjection.Scale(1.0/1.2) #scale by eta region
    
    Plot1D(PhiProjection,ipt,izt,Signal_DNN)
    
    zyam = PhiProjection.Integral(11,13)/3
    # how do I plot ZYAM, perhaps in both cases?
    #print('ZYAM=%f'%(zyam))
  
    file.Close()
    return PhiProjection

# Eta Projection

In [ ]:
def GetEtaProj(filename, ipt, izt, Signal_DNN=True):

    DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN)) #convert bool to DNN_1 (Sgn) or DNN_2 (Bkgd)
    file = ROOT.TFile(filename)
    histo2D = file.Get('DNN%i_Correlation__pT%1.0f_%1.0f__zT%1.0f_zT%1.0f' 
                       %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1],10*zTbins[izt],10*zTbins[izt+1]))
    #histo2D = file.Get('hadron_hadron_corr')
    
    print(histo2D)
    #integral
    Phi_Axis = histo2D.GetXaxis()
    EtaProjection = histo2D.ProjectionY('DNN%i_EtaProjection__pT_%1.0f_%1.0f__zt_%1.0f_%1.0f' 
                                  %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1],10*zTbins[izt],
                                    10*zTbins[izt+1]),Phi_Axis.FindBin(0.0),Phi_Axis.FindBin(math.pi/2));
    EtaProjection.SetDirectory(0)
    EtaProjection.Scale(1.0/1.2) #scale by eta region
    ntriggers = Get_NTriggers(filename, ipt, Signal_DNN)    
    EtaProjection.Scale(1.0/ntriggers) #per trigger yield
    
    Plot1D(EtaProjection,ipt,izt,Signal_DNN)
    
    #zyam = EtaProjection.Integral(11,13)/3
    # how do I plot ZYAM, perhaps in both cases?
    #print('ZYAM=%f'%(zyam))
  
    file.Close()
    return EtaProjection

# Loop and Plot Eta

In [ ]:
for ptbin in range(3):
    if(ptbin > 0): continue
    ptbin = ptbin+2
    canvases = ROOT.TCanvas("canv_phi","canv_phi",3500,2500)
    canvases.Divide(4,3)
    
    for ztbin in range(6):
    
        #grab
        hsig = GetEtaProj(RootFile,ptbin,ztbin,True) #change DNN
        hbkg = GetEtaProj(RootFile,ptbin,ztbin,False)
 
        #same scale
        ymax = 1.2*max(hsig.GetMaximum(),hbkg.GetMaximum())
        hsig.GetYaxis().SetRangeUser(0,ymax)
        hbkg.GetYaxis().SetRangeUser(0,ymax)    
    
        #Signal
        canvases.cd(ztbin*2+1)
        hsig.SetMarkerStyle(15)
        #hsig.SetMarkerSize(4)
        hsig.SetMarkerColor(4)
        hsig.Draw()
        
        #Background
        canvases.cd(ztbin*2+2)
        hbkg.SetMarkerStyle(15)
        #hbkg.SetMarkerSize(4)
        hbkg.SetMarkerColor(2)
        hbkg.Draw()
        #legend.AddEntry(hbkg,"Low DNN","l")
        #legend.Draw("same")
    
    canvases.Draw()
    canvases.SaveAs("pics/13d_Large_Eta_Corrected_ISO_Projections_Trigger_pt_%i.png"%(ptbin))

In [ ]:
RootFile2 = '13d_Same_Mix_Ratio_v1.root'
legend = [3]
for ptbin in range(3):
    if(ptbin > 0): continue
    ptbin = ptbin+2
    canvases = ROOT.TCanvas("canv_phi","canv_phi",3500,2500)
    canvases.Divide(3,2)
    legend[ptbin-2] = ROOT.TLegend(0.74,0.78,0.89,0.88)
    for ztbin in range(6):
    
        #grab
        hsig = GetEtaProj(RootFile,ptbin,ztbin,True) #no subtraction, change DNN
        hbkg = GetEtaProj(RootFile,ptbin,ztbin,False)
        #hsig2 = GetEtaProj(RootFile2,ptbin,ztbin,True)

        
        #same scale
        ymax = 1.2*max(hsig.GetMaximum(),hbkg.GetMaximum())
        hsig.GetYaxis().SetRangeUser(0,ymax)
        hbkg.GetYaxis().SetRangeUser(0,ymax)    
    
        #Signal
        canvases.cd(ztbin+1)
        hsig.SetMarkerStyle(15)
        #hsig.SetMarkerSize(4)
        hsig.SetMarkerColor(4)
        hsig.Draw()
        #hsig2.SetMarkerStyle(15)
        #hsig2.SetMarkerColor(3)
        #hsig2.SetLineColor(3)
        #hsig2.Draw("same")
        if(ztbin < 1):
            legend[ptbin-2].AddEntry(hsig, "13d","l")
            #legend[ptbin-2].AddEntry(hsig2, "13d","l")
        legend[ptbin-2].Draw()
        #SetOwnership(legend, 0)
        
    
    canvases.Draw()   
    #canvases.SaveAs("pics/13fd_overlay_Eta_Same_Mix_Projections_Trigger_pt_%i.png"%(ptbin))